In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import os
import time
import re
from urllib.parse import quote, unquote

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

### Downloading books from www.ataun.net/BIBLIOTECAGRATUITA/indice.html

In [4]:
response = requests.get('http://www.ataun.net/BIBLIOTECAGRATUITA/indice.html')
soup_ataun = BeautifulSoup(response.text, "lxml")

In [5]:
list_ataun = soup_ataun.find_all("a")[305:]
len(list_ataun)

1048

### Collecting links and downloading books

In [6]:
all_links = []
for link in list_ataun:
    if link.has_attr('href'):
        all_links.append(quote(link['href']).replace("2520","20").replace("%83%C2",""))
#unquote(all_links[0]).split("/")[-1].replace(" ", "\ ")

In [7]:
def download_pdf(url):
    options = webdriver.ChromeOptions()
    profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # Disable Chrome's PDF Viewer
                    "download.extensions_to_open": "applications/pdf"}
    options.add_experimental_option("prefs", profile)
    driver = webdriver.Chrome("/home/pnietoramos/chromedriver", chrome_options=options)  # Optional argument, if not specified will search path.
    driver.get("http://www.ataun.net/BIBLIOTECAGRATUITA/" + url)
    time.sleep(15)
    driver.close()


In [71]:
for url in all_links:
    download_pdf(url)

### Changing file names and converting to txt files

Renaming all files as author\_name\_book.pdf

In [8]:
for url in all_links:
    old_name = "'" + unquote(url).split("/")[-1] + "'"
    new_name = "_".join(unquote(url).replace(" ","_").replace(",","_").replace("-","_").split("/")[-2:])
    os.system("mv " + old_name + " " + new_name)

Converting the files to text using pdf2txt.py module

In [11]:
for url in all_links:
    file = "_".join(unquote(url).replace(" ","_").replace(",","_").replace("-","_").split("/")[-2:])
    os.system("pdf2txt.py -o " + file[:-4] + ".txt " + file)